In [4]:
import os, json
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util

load_dotenv()
uri = os.getenv("MONGODB_URI")
database = os.getenv("MONGODB_DATABASE")

client = MongoClient(uri, tz_aware=True)
coll = client[database]["pump_inventories"]

def verify_qr_chain(code: str):
    doc = coll.find_one({"current_form_qr_code": code})
    if not doc:
        return {"current_form_qr_code": code, "exists": False, "previous_present_for_same_pump": None, "details": "no document with this current_form_qr_code"}
    prev = (doc.get("previous_form_qr_code") or "").strip()
    if not prev:
        return {"current_form_qr_code": code, "exists": True, "previous_present_for_same_pump": False, "details": "document has empty previous_form_qr_code"}
    prev_doc = coll.find_one({
        "current_form_qr_code": prev,
        "location": doc.get("location"),
        "dispenser_name": doc.get("dispenser_name"),
        "pump_id": doc.get("pump_id")
    })
    return {
        "current_form_qr_code": code,
        "exists": True,
        "previous_present_for_same_pump": bool(prev_doc),
        "previous_form_qr_code": prev,
        "doc_brief": {
            "id": doc.get("id"),
            "location": doc.get("location"),
            "dispenser_name": doc.get("dispenser_name"),
            "pump_id": doc.get("pump_id"),
            "date": doc.get("date")
        },
        "previous_doc_brief": None if not prev_doc else {
            "id": prev_doc.get("id"),
            "current_form_qr_code": prev_doc.get("current_form_qr_code"),
            "location": prev_doc.get("location"),
            "dispenser_name": prev_doc.get("dispenser_name"),
            "pump_id": prev_doc.get("pump_id"),
            "date": prev_doc.get("date")
        }
    }

current_code = "11458999"
result = verify_qr_chain(current_code)
print(json.dumps(result, default=json_util.default, ensure_ascii=False, indent=2))


{
  "current_form_qr_code": "11458999",
  "exists": false,
  "previous_present_for_same_pump": null,
  "details": "no document with this current_form_qr_code"
}
